In [231]:
import numpy
import scipy

<h1 align="center">🎵 FLAC 🎵</h1>

Présenté par Alix ANNERAUD et Haijiao

# ✅ Caractéristiques

- **Type de données**: 🎵
- **Extension du fichier**: `.flac`
- **Développé par**: Josh Coalson
- **Première version**: 20 juillet 2001
- **Échantillonnage**: variable
- **Compression**: ✅
- **Perte de données**: ❌ 

# 📈 Numérisation du signal 

```mermaid
flowchart LR
    Signal_analogique[Signal analogique]
    Signal_numérique[Signal numérique]
    Signal_encodé[Signal encodé]

    Signal_analogique -->|Echantillonnage| Signal_numérique
    Signal_numérique -->|Encodage| Signal_encodé

    Signal_encodé -->|Décodage| Signal_numérique
    Signal_numérique -->|Reconstruction| Signal_analogique
```

- **Signal analogique**: Signal sonore continu dans le temps (exemple : vinyle).
- **Signal numérique**: Signal discret en temps et en amplitude (exemple : fichier WAV).
- **Signal encodé**: Signal numérique compressé (exemple : fichier FLAC, MP3, AAC, etc.).

# 🔄 Principe d'encodage

## 🔪 Division en blocs

Le signal audio est divisé en blocs contenant chacun un nombre identique d'échantillons.


```mermaid
graph TB
    Signal_audio[Signal audio]

    Signal_audio -->Canal_1[Canal 1]
    Signal_audio -->Canal_2[Canal 2]
    Signal_audio -->Canal_...[Canal ...]

    Canal_1 -->Bloc_11[Bloc 1]
    Canal_1 -->Bloc_12[Bloc 2]
    Canal_1 -->Bloc_1...[Bloc ...]

    Canal_2 -->Bloc_21[Bloc 1]
    Canal_2 -->Bloc_22[Bloc 2]
    Canal_2 -->Bloc_2...[Bloc ...]

    Canal_... -->Bloc_...1[Bloc 1]
    Canal_... -->Bloc_...2[Bloc 2]
    Canal_... -->Bloc_......[Bloc ...]
```

## 🔍 Décorrelation inter-canal (optionnel)

Dans le cas d'un signal stéréo, les canaux gauche et droit sont décorrelés en deux canaux dit "latéral" et "central".

$$
\begin{align*}
    x_\text{central} &= \frac{x_\text{gauche} + x_\text{droite}}{2} \\
    x_\text{latéral} &= x_\text{gauche} - x_\text{droite}
\end{align*}
$$

Réduit la redondance : si les canaux gauche et droit sont identiques, le canal latéral sera null.

In [245]:
def decorreller(signal):
    signal_central = signal.mean(axis=0)
    signal_lateral = signal[0] - signal[1]

    return (signal_central, signal_lateral)
    
signal = numpy.random.randint(0, 10, (2, 10))
signal_central, signal_lateral = decorreller(signal)

print(f"Signal gauche: {signal[0]}")
print(f"Signal droite: {signal[1]}")
print(f"Signal central: {signal_central}")
print(f"Signal latéral: {signal_lateral}")


Signal gauche: [7 3 2 2 7 0 9 5 4 9]
Signal droite: [8 8 7 0 6 3 7 1 7 7]
Signal central: [7.5 5.5 4.5 1.  6.5 1.5 8.  3.  5.5 8. ]
Signal latéral: [-1 -5 -5  2  1 -3  2  4 -3  2]


## 📐 Modélisation

On cherche à approximer le signal avec une fonction:

$$
x_n = f(x_{n-1}, x_{n-2}, \ldots, x_{n-p}) + e_n
$$

Où:
- $x_n$ : échantillon à l'instant $n$
- $f$ : fonction de prédiction
- $e_n$ : les résidus
- $p$ : ordre de la prédiction

Le but étant de trouver la fonction $f$ qui minimisent $e_n$ afin de réduire la quantité d'information à stocker. Les deux méthodes utilisés par le FLAC sont:

- Ajustement de polynômes simples
    - $f(x) = a_0 + a_1 x_{n-1} + a_2 x_{n-2} + \ldots + a_p x_{n-p}$
    - Rapide mais moins efficace
- Codage prédictif linéaire
    - $f(x) = \sum_{i=1}^p a_i x_{n-i}$
    - Plus complexe mais plus efficace

### Ajustement de polynômes simples

$$
\begin{align*}
    a_0 &= \frac{\sum x_n}{N} \\
    a_1 &= \frac{\sum x_n x_{n-1}}{\sum x_{n-1}^2} \\
    a_2 &= \frac{\sum x_n x_{n-2}}{\sum x_{n-2}^2} \\
    &\vdots \\
    a_p &= \frac{\sum x_n x_{n-p}}{\sum x_{n-p}^2}
\end{align*}
$$

### 🔮 Codage prédictif linéaire

On peut représenter les résidus $e_n$ comme un processus AR(p) :

$$
    R_i = \sum_{j=1}^{p} a_j R_{i-j}
$$

Où :
- $R_i$ est la fonction d'autocorrélation
- $a_j$ sont les coefficients de prédiction
- $p$ est l'ordre de prédiction

Résolu par la méthode de **Levinson-Durbin**.

In [233]:
def codage_predictif_lineaire(signal, ordre):
    autocorrelation = numpy.correlate(signal, signal, mode='full')
    autocorrelation = autocorrelation[len(autocorrelation)//2:]
   
    R = scipy.linalg.toeplitz(autocorrelation[:ordre])
    r = autocorrelation[1:ordre+1]
    coefficients = scipy.linalg.solve_toeplitz((R[:, 0], R[0, :]), r)

    return coefficients
    
def prediction_lineaire(signal, ordre):
    prediction = numpy.zeros_like(signal)
    for i in range(ordre, len(signal)):
        prediction[i] = numpy.dot(coefficients, signal[i-ordre:i][::-1])
    
    return prediction

signal = numpy.random.randint(0, 65535, 4000)  # Signal aléatoire de 10 échantillons

coefficients = codage_predictif_lineaire(signal, 10)
predictions = prediction_lineaire(signal, 10)
residus = signal - predictions

print(f'Coefficients: {coefficients}')
print(f'Signal: {signal} (valeurs uniques: {len(numpy.unique(signal))})')
print(f'Prédictions: {predictions}')
print(f'Résidus: {residus} (valeurs uniques: {len(numpy.unique(residus))})')

Coefficients: [0.09452747 0.10225915 0.09023658 0.08651685 0.10333994 0.08573053
 0.13947313 0.08106751 0.07917986 0.10597255]
Signal: [50335  9415 64657 ... 59923 41566 44234] (valeurs uniques: 3878)
Prédictions: [    0     0     0 ... 32978 40663 40911]
Résidus: [50335  9415 64657 ... 26945   903  3323] (valeurs uniques: 3893)


## 🗜️ Compression

Les résidus sont des nombres entiers très petits (peu de bits significatifs)

Codage à longueur variable :

- **Golomb-Rice**
    - Rapide à encoder et à décoder.
    - Efficace pour les distributions de symboles géométriques ou exponentielles.
- **Huffman**
    - Plus lent à encoder et à décoder.
    - Efficace pour les distributions de symboles non uniformes.

### 🌳 Codage d'Huffman

Pour rappel, pour coder des données:

- Calculer la fréquence d'apparition de chaque symbole.
- Création d'un arbre binaire où les symboles les plus fréquents sont les plus proches de la racine.
- On attribue un code binaire à chaque symbole en parcourant l'arbre.

### 🍚 Codage de Golomb-Rice

Pour coder un nombre $N$ avec un paramètre $k$:

1. Calculer le quotient $q$ et le reste $r$ : $N = q \times 2^k + r$.
2. Coder le quotient $q$ en unaire ($q$ fois le bit 1 suivi du bit 0): $q_\text{unaire} = 1^q 0$.
3. Coder le reste $r$ en binaire sur $k$ bits: $r_\text{binaire} = \text{binaire}(r, k)$
4. Combiner les deux parties: $\text{Résultat final} = q_\text{unaire} \, r_\text{binaire}$

Le choix optimal de $k$ :

$$
k_\text{optimal} = \frac{\log_2 \left( 2 - \frac{V}{I} \right)}{\log_2 \left( 1 - \frac{V}{I} \right)}
$$

Où:
- $V$ est la quantité de valeurs.
- $I$ est la taille de l'intervalle.

In [234]:
def encoder_golomb_rice(N, k):
    d = 2 ** k
    q = N // d   # Calcul du quotient
    r = N % d    # Calcul du reste
    
    q_unaire = '1' * q + '0'         # Encodage du quotient en unaire
    r_binaire = format(r, f'0{k}b')  # Encodage du reste en binaire
    
    return q_unaire + r_binaire   # Concaténation des deux encodages

for i in range(10):
    print(f'{i:2} -> {encoder_golomb_rice(i, 3)}')

 0 -> 0000
 1 -> 0001
 2 -> 0010
 3 -> 0011
 4 -> 0100
 5 -> 0101
 6 -> 0110
 7 -> 0111
 8 -> 10000
 9 -> 10001


# 🔄 Principe de décodage du FLAC

Pour décoder un fichier FLAC, il faut :

1. Lire les métadonnées

2. Décoder les résidus

3. Appliquer la prédiction inverse

4. Reconstruire le signal audio

# 🆚 Comparaisons

Nous allons comparer les différentes caractéristiques des formats audio les plus populaires:

- **WAV**: Format audio non compressé.
- **MP3**: Ancien format audio compressé avec perte créé par le Fraunhofer Institute.
- **AAC**: Format audio compressé avec perte plus récent créé par Apple.
- **FLAC**

Il y a également d'autres formats audio comme le WMA, OGG, etc. mais nous allons nous concentrer sur les 4 formats cités ci-dessus.

## 🗜️ Efficacité de la compression

1. **AAC**: Compression avec perte (environ 75-90 %).
2. **FLAC**: Compression sans perte (environ 30-60 %).
3. **MP3**: Compression avec perte (environ 75-90 %).
4. **WAV**: Pas de compression (0 %).

## 🎧 Qualité audio

1. **WAV** et **FLAC**: Qualité audio maximale.
2. **AAC**: Qualité audio très bonne.
3. **MP3**: Qualité audio bonne.

## ⚒️ Usages

- **WAV**: Anciennement utilisé par les studios d'enregistrement.
- **FLAC**: Utilisé par les studios d'enregistrement, audio-philes, etc.
- **AAC**: Utilisé par Apple sur ses appareils.
- **MP3**: Anciennement utilisé sur les CD, les baladeurs, etc.

# 👏 Fin

Merci pour votre attention ! Cette présentation est maintenant terminée.


# 📚 Références

- Wikipedia : 
    - [FLAC](https://en.wikipedia.org/wiki/FLAC)
    - [Golomb-Rice Coding](https://en.wikipedia.org/wiki/Golomb_coding)
    - [Yule-Walker Equations](https://en.wikipedia.org/wiki/Autoregressive_model#Yule-Walker_equations)
    - [Levinson-Durbin Algorithm](https://en.wikipedia.org/wiki/Levinson_recursion)
    - [Audio File Formats](https://en.wikipedia.org/wiki/Audio_file_format)
    - [Comparison of Audio Formats](https://en.wikipedia.org/wiki/Comparison_of_audio_coding_formats)
    - [Audio Coding](https://en.wikipedia.org/wiki/Audio_coding_format)
    - [Audio Compression](https://en.wikipedia.org/wiki/Audio_compression_(data))

- Xiph.org :
    - [FLAC - Official Website](https://xiph.org/flac/)
    - [FLAC - Technical Documentation](https://xiph.org/flac/documentation_format_overview.html)
    - [FLAC - Format Specification](https://xiph.org/flac/format.html)
    - [FLAC - Source Code](https://gitlab.xiph.org/xiph/flac)
